<a href="https://colab.research.google.com/github/wtrekell/soylent-army/blob/main/colab/ai_vs_human_v2-c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cell 0.1 with have an error to ⚠ Restart to reload dependencies**

1. Run Cell 0.1a
2. Error
3. Restart session
4. Sellect Cell 0.1b
2. Run cell and below  (in runtime menu)


In [ ]:
# Cell 0.1a — Dependencies (Colab setup)

%pip install -q "pandas" "numpy" "beautifulsoup4" "markdown-it-py[linkify]" "lxml" \
                "scikit-learn" "sentence-transformers" "transformers" "torch" \
                "textstat" "tqdm" "spacy" "rapidfuzz"

!python -m spacy download en_core_web_sm

In [ ]:
# Cell 0.1b - Confirm 'restart session' worked

import pandas as pd
import sklearn
import spacy
import torch
import transformers

print("✅ All libraries imported successfully!")
print("-" * 30)
print(f"Pandas version: {pd.__version__}")
print(f"spaCy version: {spacy.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")

# You can also test the spaCy model loading
try:
    nlp = spacy.load("en_core_web_sm")
    print("\n✅ spaCy model 'en_core_web_sm' loaded successfully.")
except OSError:
    print("\n❌ spaCy model 'en_core_web_sm' not found. Make sure you downloaded it in the first cell.")

In [ ]:
# Cell 0.2 — Imports & Global Config

from __future__ import annotations

import json
import os
import random
import re
import time
import unicodedata
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import pandas as pd
import textstat
from bs4 import BeautifulSoup
from markdown_it import MarkdownIt
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED); np.random.seed(SEED)

@dataclass
class Config:
    data_dir: str = "./data"
    out_dir: str = "./output"

    patterns: dict[str, str] = None

    model_name: str = "all-MiniLM-L6-v2"
    batch_size: int = 64

    ngram_range: tuple[int,int] = (1, 3)
    max_features: int | None = 50000
    sublinear_tf: bool = True

    topk_candidates: int = 10
    weight_semantic: float = 0.5
    weight_tfidf: float = 0.5

    percentile_minor: float = 0.60
    percentile_unchanged: float = 0.85
    new_floor: float = 0.25
    new_floor_tfidf: float = 0.20

    USE_SPACY: bool = True
    COMPUTE_SENT_PERPLEXITY: bool = False  # per-sentence perplexity (costly)

    ppl_model_name: str = "gpt2"
    ppl_stride: int = 512

cfg = Config(patterns={"draft":"draft*.md","refined":"refined*.md","edited":"edited*.md","final":"final*.md"})
print("v2 config loaded.")

In [ ]:
# Cell 1.1 — File Discovery (user-paste path, single or batch)


def _has_article_files(folder: Path) -> bool:
    # “Any of the 4” is enough to treat as an article folder
    return any(folder.glob(pat) for pat in cfg.patterns.values())

in_path = input("📁 Paste path to an article folder OR a parent directory: ").strip()
base = Path(in_path).expanduser().resolve()
if not base.exists():
    raise FileNotFoundError(f"Path not found: {base}")

# 1) Prefer the base directory itself, if it contains any of the 4 files
if _has_article_files(base):
    articles = [base]
    print(f"Detected article files in base → using SINGLE article: {base}")

# 2) Otherwise, look one level down for subfolders that contain any of the 4 files
else:
    candidates = [p for p in base.iterdir() if p.is_dir() and _has_article_files(p)]
    if candidates:
        articles = sorted(candidates)
        print(f"No article files in base. Using {len(articles)} subfolder article(s):")
        for a in articles: print(" -", a)
    else:
        # Nothing matched anywhere
        patterns_list = ", ".join(cfg.patterns.values())
        raise FileNotFoundError(
            f"No article files found.\n"
            f"Looked for [{patterns_list}] in:\n"
            f" - {base}\n"
            f" - its immediate subfolders"
        )

In [ ]:
# Cell 1.2 — Markdown Read + Cleanup (md → text)

md = MarkdownIt("commonmark", {})

def md_to_text(md_str: str) -> str:
    html = md.render(md_str or "")
    soup = BeautifulSoup(html, "lxml")
    text = soup.get_text(" ")
    return re.sub(r"\s+", " ", text).strip()

def read_first_match(folder: Path, pattern: str) -> str | None:
    files = sorted(folder.glob(pattern))
    if not files:
        return None
    return files[0].read_text(encoding="utf-8", errors="ignore")

In [ ]:
# Cell 1.3 — Sentence Segmentation

def split_sentences(text: str) -> list[str]:
    if not text:
        return []
    text = re.sub(r"\s+", " ", text).strip()
    parts = re.split(r"(?<=[.!?])\s+(?=[\(\"\'A-Z0-9])", text)
    return [p.strip() for p in parts if p and not p.isspace()]

In [ ]:
# Cell 1.4 — Tokenization Utilities

def tokenize(text: str) -> list[str]:
    if not text:
        return []
    text = unicodedata.normalize("NFKC", text).lower()
    text = re.sub(r"[^\w\s]", " ", text, flags=re.UNICODE)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

In [ ]:
# Cell 2.1 — TF-IDF Helpers

def fit_tfidf(corpus: list[str]) -> tuple[TfidfVectorizer, any]:
    vec = TfidfVectorizer(ngram_range=cfg.ngram_range, max_features=cfg.max_features, sublinear_tf=cfg.sublinear_tf)
    mat = vec.fit_transform(corpus)
    return vec, mat

def tfidf_matrix(vec: TfidfVectorizer, texts: list[str]):
    return vec.transform(texts)

In [ ]:
# Cell 2.2 — Embeddings

_EMB_MODEL: SentenceTransformer | None = None

def get_embedder() -> SentenceTransformer:
    global _EMB_MODEL
    if _EMB_MODEL is None:
        print(f"Loading embedding model: {cfg.model_name}")
        _EMB_MODEL = SentenceTransformer(cfg.model_name)
    return _EMB_MODEL

def embed_sentences(sents: list[str]) -> np.ndarray:
    model = get_embedder()
    embs = []
    for i in tqdm(range(0, len(sents), cfg.batch_size), desc="Embedding", unit="batch"):
        batch = sents[i:i+cfg.batch_size]
        embs.append(model.encode(batch, show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True))
    return np.vstack(embs) if embs else np.zeros((0, 384), dtype=np.float32)  # 384 for MiniLM; safe default


In [ ]:
# Cell 2.3 — Perplexity

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

_PPL = {"model": None, "tok": None}

def get_ppl_model():
    if _PPL["model"] is None:
        print(f"Loading perplexity model: {cfg.ppl_model_name}")
        _PPL["tok"] = AutoTokenizer.from_pretrained(cfg.ppl_model_name)
        _PPL["model"] = AutoModelForCausalLM.from_pretrained(cfg.ppl_model_name)
        _PPL["model"].eval()
    return _PPL["model"], _PPL["tok"]

@torch.no_grad()
def calc_perplexity(text: str) -> float:
    model, tokenizer = get_ppl_model()
    if not text.strip():
        return 0.0
    enc = tokenizer(text, return_tensors='pt')
    input_ids = enc.input_ids
    seq_len = input_ids.size(1)
    max_len = getattr(model.config, "n_positions", 1024)
    stride = cfg.ppl_stride

    nlls = []
    prev_end = 0
    for begin in range(0, seq_len, stride):
        end = min(begin + max_len, seq_len)
        trg_len = end - prev_end
        input_chunk = input_ids[:, begin:end]
        target_ids = input_chunk.clone()
        target_ids[:, :-trg_len] = -100
        outputs = model(input_chunk, labels=target_ids)
        nlls.append(outputs.loss * trg_len)
        prev_end = end
        if end == seq_len:
            break
    ppl = torch.exp(torch.stack(nlls).sum() / seq_len)
    return float(ppl.item())

def calc_sentence_ppl(sentences: list[str]) -> list[float]:
    if not cfg.COMPUTE_SENT_PERPLEXITY:
        return [None] * len(sentences)
    return [calc_perplexity(s) for s in tqdm(sentences, desc="Per-sentence PPL")]

In [ ]:
# Cell 2.4 — Burstiness & Fano

def sentence_token_lengths(sentences: list[str], tokenizer=None) -> list[int]:
    # use embedding tokenization surrogate: whitespace word count (robust & fast)
    return [len(tokenize(s)) for s in sentences]

def calc_burstiness(sentences: list[str]) -> float:
    lens = sentence_token_lengths(sentences)
    return float(np.std(lens)) if lens else 0.0

def calc_fano(sentences: list[str]) -> float:
    lens = sentence_token_lengths(sentences)
    if not lens:
        return 0.0
    mean = np.mean(lens)
    var = np.var(lens)
    return float(var / mean) if mean > 0 else 0.0

In [ ]:
# Cell 2.5 — Stylometrics & Readability

def type_token_ratio(text: str) -> float:
    toks = tokenize(text)
    return float(len(set(toks)) / max(1, len(toks)))

_SPACY_NLP = None
def get_spacy():
    global _SPACY_NLP
    if not cfg.USE_SPACY:
        return None
    if spacy is None:
        print("spaCy not available. Set USE_SPACY=False or install spaCy + en_core_web_sm.")
        return None
    if _SPACY_NLP is None:
        _SPACY_NLP = spacy.load("en_core_web_sm")
    return _SPACY_NLP

def pos_ratios(text: str) -> dict[str, float]:
    nlp = get_spacy()
    if nlp is None:
        return {}
    doc = nlp(text)
    counts = {}
    total = 0
    for t in doc:
        if t.is_space: continue
        pos = t.pos_
        counts[pos] = counts.get(pos, 0) + 1
        total += 1
    return {k: v/total for k, v in counts.items()} if total else {}

def punctuation_rates(text: str) -> dict[str, float]:
    total = max(1, len(text))
    return {
        "comma_rate": text.count(",")/total,
        "semicolon_rate": text.count(";")/total,
        "colon_rate": text.count(":")/total,
    }

def readability_scores(text: str) -> dict[str, float]:
    # textstat guards internally; returns 0 if cannot compute
    try:
        return {
            "flesch_reading_ease": float(textstat.flesch_reading_ease(text)),
            "flesch_kincaid_grade": float(textstat.flesch_kincaid_grade(text)),
            "gunning_fog": float(textstat.gunning_fog(text)),
        }
    except Exception:
        return {}

def compute_stylometrics(text: str, sentences: list[str]) -> dict[str, float]:
    toks = tokenize(text)
    lens = [len(tokenize(s)) for s in sentences] if sentences else []
    base = {
        "ttr": type_token_ratio(text),
        "avg_word_len": (sum(len(t) for t in toks)/len(toks)) if toks else 0.0,
        "avg_sent_len_tokens": (np.mean(lens) if lens else 0.0),
        "var_sent_len_tokens": (np.var(lens) if lens else 0.0),
    }
    base.update(punctuation_rates(text))
    base.update(readability_scores(text))
    base.update(pos_ratios(text))
    return base

In [ ]:
# Cell 3.1 — Version Loader

VERSION_ORDER = ["draft","refined","edited","final"]

def load_versions(article_folder: Path) -> dict[str, str | None]:
    out = {}
    for v in VERSION_ORDER:
        pat = cfg.patterns.get(v)
        out[v] = read_first_match(article_folder, pat) if pat else None
    return out

In [ ]:
# Cell 3.2 — Sentence Banks

def to_sentences(md_content: str | None) -> list[str]:
    return split_sentences(md_to_text(md_content or ""))

def build_sentence_banks(versions: dict[str,str]) -> tuple[list[str], list[str], list[tuple[str,int]]]:
    sents = {v: to_sentences(versions.get(v)) for v in VERSION_ORDER}
    final_sents = sents["final"] or []
    candidates, cand_meta = [], []
    for v in ["draft","refined","edited"]:
        for idx, s in enumerate(sents.get(v) or []):
            candidates.append(s); cand_meta.append((v, idx))
    return final_sents, candidates, cand_meta

In [ ]:
# Cell 3.3 — Fast Similarities

def fast_similarities(final_sents: list[str], candidates: list[str]):
    if not final_sents or not candidates:
        return None, None, None

    tfidf_corpus = final_sents + candidates
    vec, mat = fit_tfidf(tfidf_corpus)
    tfidf_final = mat[:len(final_sents)]
    tfidf_cands = mat[len(final_sents):]
    sim_tfidf = cosine_similarity(tfidf_final, tfidf_cands)

    emb_final = embed_sentences(final_sents)
    emb_cands = embed_sentences(candidates)
    sim_sem = cosine_similarity(emb_final, emb_cands)

    combined_fast = cfg.weight_semantic * sim_sem + cfg.weight_tfidf * sim_tfidf
    return combined_fast, sim_sem, sim_tfidf

In [ ]:
# Cell 3.4 — Shortlist Top-K

def shortlist_indices(combined_fast: np.ndarray, k: int) -> list[np.ndarray]:
    k = max(1, min(k, combined_fast.shape[1])) if combined_fast is not None else 1
    idx_list = []
    for i in range(combined_fast.shape[0]):
        row = combined_fast[i]
        take = np.argpartition(-row, kth=k-1)[:k]
        idx_list.append(take)
    return idx_list


In [ ]:
# Cell 4.1 — Heavy Metrics (Jaccard + Levenshtein)

def jaccard_tokens(a: str, b: str) -> float:
    A, B = set(tokenize(a)), set(tokenize(b))
    if not A and not B: return 1.0
    if not A or not B:  return 0.0
    return len(A & B) / float(len(A | B))

def lev_ratio(a: str, b: str) -> float:
    # normalized similarity in [0,1]
    if not a and not b: return 1.0
    max_len = max(len(a), len(b)) or 1
    dist = Levenshtein.distance(a, b)
    return 1.0 - (dist / max_len)

In [ ]:
# Cell 4.2 — Final Scoring & Attribution

def score_and_attribute(final_sents, candidates, cand_meta, combined_fast, sim_sem, sim_tfidf):
    rows = []
    heavy_calls = 0
    if combined_fast is None or combined_fast.size == 0:
        # No candidates; mark all as "new/major"
        for i, f in enumerate(final_sents):
            rows.append({
                "final_index": i,
                "final_sentence": f,
                "origin_version": "none",
                "origin_index": None,
                "origin_sentence": None,
                "semantic_sim": None,
                "tfidf_sim": None,
                "jaccard": None,
                "levenshtein": None,
                "combined_sim": 0.0,
                "modification_label": "major"
            })
        return rows, heavy_calls

    topk_idx = shortlist_indices(combined_fast, cfg.topk_candidates)

    for i, f in enumerate(final_sents):
        cand_ids = topk_idx[i]
        best = None
        best_score = -1.0
        for ci in cand_ids:
            c = candidates[ci]
            # lexical_avg = mean(jaccard, lev_ratio, tfidf_sim)
            jac = jaccard_tokens(f, c)
            lev = lev_ratio(f, c)
            tf  = float(sim_tfidf[i, ci])
            lexical_avg = (jac + lev + tf) / 3.0
            sem = float(sim_sem[i, ci])
            score = 0.5*sem + 0.5*lexical_avg
            heavy_calls += 1
            if score > best_score:
                best_score = score
                best = (ci, jac, lev, tf, sem, score)

        if best is None:
            rows.append({
                "final_index": i,
                "final_sentence": f,
                "origin_version": "none",
                "origin_index": None,
                "origin_sentence": None,
                "semantic_sim": None,
                "tfidf_sim": None,
                "jaccard": None,
                "levenshtein": None,
                "combined_sim": 0.0,
                "modification_label": "major"
            })
        else:
            ci, jac, lev, tf, sem, score = best
            o_v, o_i = cand_meta[ci]
            rows.append({
                "final_index": i,
                "final_sentence": f,
                "origin_version": o_v,
                "origin_index": int(o_i),
                "origin_sentence": candidates[ci],
                "semantic_sim": float(sem),
                "tfidf_sim": float(tf),
                "jaccard": float(jac),
                "levenshtein": float(lev),
                "combined_sim": float(score),
                "modification_label": None  # filled next cell
            })

    return rows, heavy_calls

In [ ]:
# Cell 4.3 — Labeling by Percentiles (+ “new” rule)

def label_rows(rows: list[dict]) -> None:
    scores = np.array([r["combined_sim"] for r in rows], dtype=float)
    if scores.size == 0:
        return
    p60 = float(np.percentile(scores, cfg.percentile_minor*100))
    p85 = float(np.percentile(scores, cfg.percentile_unchanged*100))
    for r in rows:
        score = r["combined_sim"]
        tfidf = (r["tfidf_sim"] or 0.0)
        if score < cfg.new_floor and tfidf < cfg.new_floor_tfidf:
            r["modification_label"] = "major"  # effectively “new”/major
        elif score >= p85:
            r["modification_label"] = "unchanged"
        elif score >= p60:
            r["modification_label"] = "minor"
        else:
            r["modification_label"] = "major"

In [ ]:
# Cell 4.4 — Summaries & Doc Metrics

def distributions_from_rows(rows: list[dict]) -> tuple[dict,str]:
    df = pd.DataFrame(rows)
    if df.empty:
        origin_dist = dict.fromkeys(["draft", "refined", "edited", "none"], 0.0)
        mod_dist = dict.fromkeys(["unchanged", "minor", "major"], 0.0)
        return origin_dist, mod_dist

    origin_dist = (df["origin_version"].fillna("none")
                   .value_counts(normalize=True)
                   .reindex(["draft","refined","edited","none"], fill_value=0.0)
                   .to_dict())
    mod_dist = (df["modification_label"].fillna("major")
                .value_counts(normalize=True)
                .reindex(["unchanged","minor","major"], fill_value=0.0)
                .to_dict())
    return origin_dist, mod_dist

def doc_metrics(text_final: str, final_sents: list[str]) -> dict:
    return {
        "perplexity": calc_perplexity(text_final),
        "burstiness_std": calc_burstiness(final_sents),
        "fano_factor": calc_fano(final_sents),
        "stylometrics": compute_stylometrics(text_final, final_sents),
    }

In [ ]:
# Cell 4.5 — Analyzer (per article) + Diagnostics

def analyze_article(article_path: Path):
    t0 = time.perf_counter()

    versions = load_versions(article_path)
    final_text = md_to_text(versions.get("final") or "")
    final_sents, candidates, cand_meta = build_sentence_banks(versions)

    if not final_sents:
        print(f"[SKIP] No final sentences: {article_path}")
        return None

    combined_fast, sim_sem, sim_tfidf = fast_similarities(final_sents, candidates)
    rows, heavy_calls = score_and_attribute(final_sents, candidates, cand_meta,
                                            combined_fast, sim_sem, sim_tfidf)
    label_rows(rows)
    origin_dist, mod_dist = distributions_from_rows(rows)

    metrics = doc_metrics(final_text, final_sents)
    if cfg.COMPUTE_SENT_PERPLEXITY:
        sent_ppl = calc_sentence_ppl(final_sents)
        for r, ppl in zip(rows, sent_ppl, strict=False):
            r["perplexity"] = ppl

    summary = {
        "origin_distribution": origin_dist,
        "modification_distribution": mod_dist,
        "counts": {
            "sentences_final": int(len(final_sents)),
            "heavy_comparisons": int(heavy_calls),
            "avg_heavy_per_final": float(heavy_calls / max(1, len(final_sents))),
            "topk": int(cfg.topk_candidates)
        }
    }

    payload = {
        "version": "2.0-analysis",
        "article_name": article_path.name,
        "config": {
            "model_name": cfg.model_name,
            "topk_candidates": cfg.topk_candidates,
            "weights": {"semantic": cfg.weight_semantic, "tfidf": cfg.weight_tfidf},
            "percentiles": {"minor": cfg.percentile_minor, "unchanged": cfg.percentile_unchanged},
            "new_floor": {"combined": cfg.new_floor, "tfidf": cfg.new_floor_tfidf},
        },
        "summary": summary,
        "doc_metrics": metrics,
        "rows": rows,
        # Legacy block for backward compatibility (thin but enough for old visualizers)
        "attribution_analysis": {
            "statistics": {
                "origin_distribution": origin_dist,
                "modification_distribution": mod_dist
            }
        }
    }

    print(f"\nAnalysis: {article_path.name}")
    print(f"- Final sentences analyzed: {len(final_sents)}")
    print(f"- Heavy comparisons executed: {heavy_calls} (expected ≈ topK={cfg.topk_candidates} per final)")
    print(f"- Avg heavy comps per final: {summary['counts']['avg_heavy_per_final']:.2f}")
    print(f"- Unchanged/minor/major (%): "
          f"{mod_dist.get('unchanged',0):.2%} / {mod_dist.get('minor',0):.2%} / {mod_dist.get('major',0):.2%}")

    t1 = time.perf_counter()
    footer = {"elapsed_sec": float(t1 - t0)}
    return payload, footer

In [ ]:
# Cell 4.6 — Integrate Optional Detectors into Results

def integrate_probs(rows: list[dict], final_text: str):
    # rows must already exist (from score_and_attribute / label_rows)
    sentences = [r["final_sentence"] for r in rows]
    probs = apply_detectors_to_rows(sentences, final_text)
    # Attach per-sentence if available
    for name, arr in probs.items():
        if arr is None:
            continue
        for r, p in zip(rows, arr, strict=False):
            r[name] = float(p)
    # Return doc-level summaries
    doc_levels = {}
    if probs.get("p_ai_feature") is not None:
        doc_levels["p_ai_feature"] = float(np.mean(probs["p_ai_feature"]))
    if probs.get("p_ai_transformer") is not None:
        doc_levels["p_ai_transformer"] = float(np.mean(probs["p_ai_transformer"]))
    if probs.get("p_ai_ensemble") is not None:
        doc_levels["p_ai"] = float(np.mean(probs["p_ai_ensemble"]))
    return doc_levels

In [ ]:
# Cell 5.1 — Feature-Engineered Detector (sklearn; optional)

# 🛑 will occur unless we install and train our own models.

import pickle

_FE = {"vec": None, "model": None}

def _try_load_fe_artifacts(model_dir: Path = Path("./models")):
    try:
        vec_path = model_dir / "fe_vectorizer.pkl"
        mdl_path = model_dir / "fe_model.pkl"
        if vec_path.exists() and mdl_path.exists():
            with open(vec_path, "rb") as f: _FE["vec"] = pickle.load(f)
            with open(mdl_path, "rb") as f: _FE["model"] = pickle.load(f)
            print(f"[FE] Loaded vectorizer+model from {model_dir}")
        else:
            print("🛑 Artifacts not found; skipping feature-engineered detector.")
    except Exception as e:
        print(f"[FE] Failed to load artifacts: {e}")

def _fe_features(texts: list[str], doc_text: str) -> np.ndarray:
    if _FE["vec"] is None:
        return None
    X = _FE["vec"].transform(texts)  # TF-IDF / ngram features inside the vectorizer
    return X

def fe_predict_proba(sentences: list[str], doc_text: str) -> np.ndarray | None:
    if _FE["model"] is None:
        return None
    X = _fe_features(sentences, doc_text)
    if X is None:
        return None
    mdl = _FE["model"]
    # Support both probabilistic and decision-function models
    if hasattr(mdl, "predict_proba"):
        proba = mdl.predict_proba(X)
        # assume binary [human, ai]; return ai column
        return proba[:, 1]
    elif hasattr(mdl, "decision_function"):
        scores = mdl.decision_function(X)
        # map decision scores to (0,1) via logistic
        return 1.0 / (1.0 + np.exp(-scores))
    else:
        # fallback to labels {0,1}
        preds = mdl.predict(X)
        return preds.astype(float)

_try_load_fe_artifacts()

In [ ]:
# Cell 4.1 — collect_doc_metrics()

import shutil
from pathlib import Path

import torch
from transformers import AutoModelForSequenceClassification

_TR = {"tok": None, "mdl": None, "id2label": None}

PRIMARY_MODEL  = "fakespot-ai/roberta-base-ai-text-detection-v1"   # recommended
FALLBACK_MODEL = "Hello-SimpleAI/chatgpt-detector-roberta"         # HC3-based
MODEL_DIR = Path("./models/roberta-detector")

def _download_and_save(model_id: str, target: Path) -> bool:
    try:
        print(f"⏬ Attempting to download: {model_id}")
        tok = AutoTokenizer.from_pretrained(model_id)
        mdl = AutoModelForSequenceClassification.from_pretrained(model_id)
        if target.exists():
            shutil.rmtree(target)
        target.mkdir(parents=True, exist_ok=True)
        tok.save_pretrained(target)
        mdl.save_pretrained(target)
        print(f"✅ Saved detector to {target} ({model_id})")
        return True
    except Exception as e:
        print(f"❌ Failed to download {model_id}: {e}")
        return False

def _ensure_checkpoint():
    if MODEL_DIR.exists() and any(MODEL_DIR.iterdir()):
        return True
    ok = _download_and_save(PRIMARY_MODEL, MODEL_DIR)
    if not ok:
        ok = _download_and_save(FALLBACK_MODEL, MODEL_DIR)
    if not ok:
        print("⚠ No detector downloaded. Transformer-based detection will noop gracefully.")
    return ok

def _try_load_transformer(model_dir: Path = MODEL_DIR):
    try:
        if not model_dir.exists() or not any(model_dir.iterdir()):
            if not _ensure_checkpoint():
                print("[TR] Transformer checkpoint not found; skipping transformer detector.")
                return
        _TR["tok"] = AutoTokenizer.from_pretrained(model_dir)
        _TR["mdl"] = AutoModelForSequenceClassification.from_pretrained(model_dir)
        _TR["mdl"].eval()
        _TR["id2label"] = _TR["mdl"].config.id2label
        print(f"[TR] Loaded transformer detector from {model_dir}")
    except Exception as e:
        print(f"[TR] Failed to load transformer detector: {e}")

@torch.no_grad()
def tr_predict_proba(sentences: list[str]) -> np.ndarray | None:
    if _TR["mdl"] is None or _TR["tok"] is None:
        return None
    tok = _TR["tok"]; mdl = _TR["mdl"]
    out = []
    bs = 32
    for i in range(0, len(sentences), bs):
        batch = sentences[i:i+bs]
        enc = tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        logits = mdl(**enc).logits
        if logits.shape[-1] == 2:
            proba = torch.softmax(logits, dim=-1)[:, 1]  # assume index 1 = AI
        else:
            # try to find label "AI" in id2label; else use last column
            if _TR["id2label"]:
                ai_index = None
                for k, v in _TR["id2label"].items():
                    if str(v).lower().startswith("ai"):
                        ai_index = int(k)
                        break
                if ai_index is None:
                    ai_index = logits.shape[-1] - 1
            else:
                ai_index = logits.shape[-1] - 1
            proba = torch.softmax(logits, dim=-1)[:, ai_index]
        out.append(proba.cpu().numpy())
    return np.concatenate(out, axis=0) if out else np.zeros((0,), dtype=float)

# Load (and auto-download if missing)
_try_load_transformer()

In [ ]:
# Cell 5.1 — Feature-Engineered Detector (sklearn)

def ensemble_probs(*arrays: np.ndarray | None) -> np.ndarray | None:
    parts = [a for a in arrays if a is not None]
    if not parts:
        return None
    # simple mean; weights can be added if desired
    return np.mean(np.vstack(parts), axis=0)

def apply_detectors_to_rows(final_sents: list[str], final_text: str) -> dict[str, np.ndarray | None]:
    p_fe  = fe_predict_proba(final_sents, final_text)
    p_tr  = tr_predict_proba(final_sents)
    p_ens = ensemble_probs(p_fe, p_tr)
    return {"p_ai_feature": p_fe, "p_ai_transformer": p_tr, "p_ai_ensemble": p_ens}

In [ ]:
# Cell 5.2 — Transformer Detector (RoBERTa/BERT)

def detectgpt_score(text: str) -> float | None:
    # Placeholder: wire to your logits-capable LM + perturbation routine when available.
    # Return a float where higher → more likely machine-generated (by that LM).
    return None

In [ ]:
# Cell 5.3 — Ensemble Wrapper

def watermark_p_value(text: str) -> float | None:
    # Placeholder: implement z-test over green-list token proportion when key is available.
    return None

In [ ]:
# Cell 6.1 — analyze_article()

def analyze_article(article_path: Path):
    t0 = time.perf_counter()

    versions = load_versions(article_path)
    final_text = md_to_text(versions.get("final") or "")
    final_sents, candidates, cand_meta = build_sentence_banks(versions)

    if not final_sents:
        print(f"[SKIP] No final sentences: {article_path}")
        return None

    combined_fast, sim_sem, sim_tfidf = fast_similarities(final_sents, candidates)
    rows, heavy_calls = score_and_attribute(final_sents, candidates, cand_meta,
                                            combined_fast, sim_sem, sim_tfidf)
    label_rows(rows)

    # OPTIONAL detectors (safe if artifacts/checkpoints absent)
    doc_prob_levels = integrate_probs(rows, final_text)

    origin_dist, mod_dist = distributions_from_rows(rows)
    metrics = doc_metrics(final_text, final_sents)

    # Optional advanced hooks
    dg = detectgpt_score(final_text)
    if dg is not None:
        metrics["detectgpt_score"] = dg
    wp = watermark_p_value(final_text)
    if wp is not None:
        metrics["watermark_pval"] = wp

    if cfg.COMPUTE_SENT_PERPLEXITY:
        sent_ppl = calc_sentence_ppl(final_sents)
        for r, ppl in zip(rows, sent_ppl, strict=False):
            r["perplexity"] = ppl

    # fold in doc-level AI probabilities if present
    if doc_prob_levels:
        metrics.update(doc_prob_levels)

    summary = {
        "origin_distribution": origin_dist,
        "modification_distribution": mod_dist,
        "counts": {
            "sentences_final": int(len(final_sents)),
            "heavy_comparisons": int(heavy_calls),
            "avg_heavy_per_final": float(heavy_calls / max(1, len(final_sents))),
            "topk": int(cfg.topk_candidates)
        }
    }

    payload = {
        "version": "2.0-analysis",
        "article_name": article_path.name,
        "config": {
            "model_name": cfg.model_name,
            "topk_candidates": cfg.topk_candidates,
            "weights": {"semantic": cfg.weight_semantic, "tfidf": cfg.weight_tfidf},
            "percentiles": {"minor": cfg.percentile_minor, "unchanged": cfg.percentile_unchanged},
            "new_floor": {"combined": cfg.new_floor, "tfidf": cfg.new_floor_tfidf},
        },
        "summary": summary,
        "doc_metrics": metrics,
        "rows": rows,
        "attribution_analysis": {  # legacy shim
            "statistics": {
                "origin_distribution": origin_dist,
                "modification_distribution": mod_dist
            }
        }
    }

    print(f"\nAnalysis: {article_path.name}")
    print(f"- Final sentences analyzed: {len(final_sents)}")
    print(f"- Heavy comparisons executed: {heavy_calls} (expected ≈ topK={cfg.topk_candidates} per final)")
    print(f"- Avg heavy comps per final: {summary['counts']['avg_heavy_per_final']:.2f}")
    print(f"- Unchanged/minor/major (%): "
          f"{mod_dist.get('unchanged',0):.2%} / {mod_dist.get('minor',0):.2%} / {mod_dist.get('major',0):.2%}")
    if doc_prob_levels:
        print("- Doc AI probability:", {k: f"{v:.2%}" for k, v in doc_prob_levels.items()})

    t1 = time.perf_counter()
    footer = {"elapsed_sec": float(t1 - t0)}
    return payload, footer

In [ ]:
# Cell 7.1 — Batch Runner

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def write_outputs(article_name: str, payload: dict, footer: dict, out_dir: Path):
    ensure_dir(out_dir)
    base = out_dir / article_name
    json_path = out_dir / f"{article_name}_complete_summary.json"
    footer_path = out_dir / f"{article_name}_footer_metrics.json"
    csv_path = out_dir / f"{article_name}_final_sentence_attribution.csv"

    # JSON
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

    # Footer
    with open(footer_path, "w", encoding="utf-8") as f:
        json.dump(footer, f, ensure_ascii=False, indent=2)

    # CSV from rows
    df_rows = pd.DataFrame(payload["rows"])
    df_rows.to_csv(csv_path, index=False, encoding="utf-8")

    return {"json": str(json_path), "csv": str(csv_path), "footer": str(footer_path)}

In [ ]:
# Cell 7.2 — Outputs Summary

results = []
out_dir = Path(cfg.out_dir)
ensure_dir(out_dir)

for art in tqdm(articles, desc="Articles", unit="article"):
    res = analyze_article(art)
    if not res:
        continue
    payload, footer = res
    paths = write_outputs(payload["article_name"], payload, footer, out_dir)
    results.append({"article_name": payload["article_name"], **paths})

if not results:
    print("\nNo outputs produced. Ensure your data folders/files match expected patterns.")
else:
    print("\nSummary of outputs:")
    for r in results:
        print(f"- {r['article_name']} -> JSON: {r['json']} | CSV: {r['csv']}")

In [ ]:
# Cell 7.3 — Collection Summary CSV

from pathlib import Path

import pandas as pd

if results:
    rows = []
    for r in results:
        with open(r["json"], encoding="utf-8") as f:
            p = json.load(f)
        s = p["summary"]; m = p["doc_metrics"]

        row = {
            "article_name":     p["article_name"],
            "sentences_final":  s["counts"]["sentences_final"],

            # Origin distributions
            "origin_draft":     s["origin_distribution"].get("draft", 0.0),
            "origin_refined":   s["origin_distribution"].get("refined", 0.0),
            "origin_edited":    s["origin_distribution"].get("edited", 0.0),
            "origin_none":      s["origin_distribution"].get("none", 0.0),

            # Modification distributions
            "mod_unchanged":    s["modification_distribution"].get("unchanged", 0.0),
            "mod_minor":        s["modification_distribution"].get("minor", 0.0),
            "mod_major":        s["modification_distribution"].get("major", 0.0),

            # Core statistical metrics
            "ppl":              m.get("perplexity", None),
            "burstiness_std":   m.get("burstiness_std", None),
            "fano_factor":      m.get("fano_factor", None),

            # Model-based AI probabilities (optional, only if detectors ran)
            "p_ai_feature":     m.get("p_ai_feature", None),      # sklearn/FE detector
            "p_ai_transformer": m.get("p_ai_transformer", None),  # RoBERTa/BERT detector
            "p_ai_ensemble":    m.get("p_ai", None),              # blended ensemble
        }
        rows.append(row)

    df = pd.DataFrame(rows).sort_values("article_name")
    path = Path(cfg.out_dir) / "_collection_summary.csv"
    df.to_csv(path, index=False, encoding="utf-8")
    print(f"\n[Collection] Wrote {len(df)} rows → {path}")
else:
    print("[Collection] No results to summarize.")

In [ ]:
import zipfile
from pathlib import Path

out_dir = Path(cfg.out_dir)
zip_filename = f"{out_dir.name}_archive.zip"
zip_path = out_dir.parent / zip_filename # Place the zip file one level up from the output directory

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Walk through the output directory and add files to the zip
    for root, _, files in os.walk(out_dir):
        for file in files:
            file_path = Path(root) / file
            # Add file to zip, preserving directory structure relative to out_dir
            zipf.write(file_path, arcname=file_path.relative_to(out_dir))

print(f"Successfully created zip archive: {zip_path}")